# Holding Analysis Tool - data extraction for BPIDs - 2020-08-06

Final report
* https://docs.google.com/document/d/1yQJZIf8X7kJrUTOvl3XlIjl74VNnMeKNIPf1CQqAT8E/edit

## Data Sources Information

### Customers IDs 

* BPIDs for prototype from Markus https://docs.google.com/spreadsheets/d/1bIRQxULsSkGrtVRd63-GcgM67NSFsScPa6VC0z5xYf0/edit#gid=0
* Site to BP ID mapping table https://hive.springernature.com/docs/DOC-185186
    * `ZSite 07-29-2020.XLSX`, then export to CSV

### APIs: BPIDs hierarchy

* api_parents = "http://institution-service.live.cf.private.springer.com/v1/hierarchy/{}"
* api_children = "http://institution-service.live.cf.private.springer.com/v1/children/{}"

### APIs: KBARTS

* url_books = "http://ml-read-online.live.sl.i.springer.com:7655/admin-portal/201802052054-134/kbart/book-report.xqy?bp={}"
* url_journals = "http://ml-read-online.live.sl.i.springer.com:7655/admin-portal/201802052054-134/kbart/journal-report.xqy?bp={}"
* url_nature = "http://entitlements-api.live.private.springernature.io/entitlements/v1/journals/kbart?bpid={}"

### APIs: Usage on GBQ

* online console https://console.cloud.google.com/bigquery?project=usage-data-reporting&organizationId=271269911051
* hive docs https://hive.springernature.com/groups/sap-business-intelligence/pages/google-bigquery-support
* query for Springer usage (using BPIDs) is:

```
Select DOI, Usage_2016, Usage_2017, Usage_2018, Denials_2016, Denials_2017, Denials_2018 FROM (
    SELECT 
    BP_ID, Item_DOI as DOI,  sum(clicks),
    sum(Case when Calendar_Year = 2016 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2018,
    sum(Case when Calendar_Year = 2016 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2018,

    FROM `usage-data-reporting.PROD_IFM_Usage.mv_usage_bp_monthly_spl` 
    Where 
    Document_Type = 'ARTICLE'
    and Item_DOI is not null
    and DOI is not null
    and clicks is not null
    and Calendar_Year in (2016, 2017, 2018)
    Group by 1, 2
    Order by 1, 2)
Where BP_ID = '2000277680' 
```
* query for Nature usage, using Site IDs, is:

```
Select DOI, Usage_2016, Usage_2017, Usage_2018, Denials_2016, Denials_2017, Denials_2018 FROM (
 
SELECT
Site_ID, Item_DOI as DOI,  sum(clicks),
 
sum(Case when Calendar_Year = 2016 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2016,
sum(Case when Calendar_Year = 2017 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2017,
sum(Case when Calendar_Year = 2018 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2018,
sum(Case when Calendar_Year = 2016 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2016,
sum(Case when Calendar_Year = 2017 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2017,
sum(Case when Calendar_Year = 2018 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2018,
 
FROM `usage-data-reporting.PROD_IFM_Usage.mv_usage_bp_monthly_nat`
Where
Item_DOI is not null
and DOI is not null
and clicks is not null
and Calendar_Year in (2016, 2017, 2018)
Group by 1, 2
Order by 1, 2)
 
Where Site_ID = '2000323035'
```

## Prerequisites

In [ ]:
import requests
import pandas as pd
import json
import os
from tqdm.notebook import tqdm as progress
from google.cloud import bigquery

from IPython.display import display

# save to an external HD
OUTPUT_FOLDER = "/Volumes/Albano/2020-08-06-holding-analysis-extraction"

## 1. Load Customer BPIDs and SITEids from CSV

In [39]:

#
# bpids
#
bpids = pd.read_csv("data_input/Markus - HA BPIDs for MVP - Sheet1.csv")
BPIDS_LIST = bpids["BPID"].to_list()
display(bpids.head())
BPIDS_LIST_UNIQUE = list(set(BPIDS_LIST))


#
# siteids
#

siteids = pd.read_csv("data_input/Site to BP ID mapping table_07-29-2020.csv")
display(siteids.head())

#
# functions
#

def gen_bpid_folder(folder, top_level_folder=OUTPUT_FOLDER, docreate=True):
    "util to create output folders for each bpid"
    if not os.path.exists(top_level_folder):
        if docreate: os.makedirs(top_level_folder)
    if not os.path.exists(top_level_folder + "/" + folder):
        if docreate: os.makedirs(top_level_folder + "/" + folder)
    return top_level_folder + "/" + folder + "/"

def bpid_to_siteid(bpid):
    "return a siteid from a bpid, if found"
    try:
        return siteids[siteids['Business Partner'] == bpid]['Identification number'].values[0]
    except:
        return None
    
def test():
    for x in sorted(BPIDS_LIST):
        print(bpid_to_siteid(x))
        
def count_bpid_mapped():
    "util for qa - count how many bpids have mappings to site IDs"
    return len([x for x in BPIDS_LIST_UNIQUE if bpid_to_siteid(x)])
    # => 154 with mappings to site IDs


print("BPIDs: ", len(BPIDS_LIST)) 
print("BPIDs unique: ", len(BPIDS_LIST_UNIQUE)) 
print("SiteIDs: ", len(siteids)) 
print("BPIDs (unique) mapped to SiteIDs: ", count_bpid_mapped())     

,BPID,Account,Contact
0,3000612920,Research Organization of Information,Martijn Timman
1,2000703964,INSERM DISC-IST,Martijn Timman
2,2000478521,The University of Tokyo,Martijn Timman
3,2000277680,Shanghai Library / Institute of,Martijn Timman
4,2000342845,National Library of China NLC,Martijn Timman


,Business Partner,Identification Type,Identification number,BP Type,Combined,New Entry?
0,3000142312,ZSITE,15760,5,3000142312 15760,No
1,3000181182,ZSITE,6763,5,3000181182 6763,No
2,3000182792,ZSITE,2772,5,3000182792 2772,No
3,3000241247,ZSITE,7573,5,3000241247 7573,No
4,3000262079,ZSITE,21218,5,3000262079 21218,No


BPIDs:  192
BPIDs unique:  189
SiteIDs:  9699
BPIDs (unique) mapped to SiteIDs:  151


## 2. Extracting bpids parents/children info and kbarts (entitlements) 

### 2.1 API for parents and children

* from a single BPID, pull out all parents and children as JSON
* save JSON to folder, if needed
* return simple list of BPIDs, including original one

PS
* `extract_related_bpids_from_local_file` is a modified version that uses previously saved local data

In [29]:
api_parents = "http://institution-service.live.cf.private.springer.com/v1/hierarchy/{}"
api_children = "http://institution-service.live.cf.private.springer.com/v1/children/{}"

        
def api_extract_related_bpids(BPID):
    BPID = str(BPID)
    print("======\nChecking Hierarchy for BPID: ", BPID, "\n======")
    path = gen_bpid_folder(BPID)
    # call API
    parents = requests.get(api_parents.format(BPID)).json()
    children = requests.get(api_children.format(BPID)).json()
    # save
    with open(path + 'parents.json', 'w') as outfile:
        json.dump(parents, outfile)
    with open(path + 'children.json', 'w') as outfile:
        json.dump(children, outfile)
    print("..data saved in", path)
    # extract BPIDs as list
    bpid_list = [BPID] # seed with main bpid
    for x in parents["parents"]: 
        bpid_list.append(x['institution']['id'])
        if x['parents']:  # just go one level up
            for z in x["parents"]: 
                bpid_list.append(z['institution']['id'])            
    for x in children:
        bpid_list.append(x['id'])
    print("Related BPIDs found: ", len(bpid_list), "=", str(bpid_list))
    return bpid_list


def extract_related_bpids_from_local_file(BPID):
    """use the previously saved json data (from extract_related_bpids) instead of calling SN API"""
    BPID = str(BPID)
    print("======\nChecking Pre-saved Hierarchy for BPID: ", BPID, "\n======")
    path = gen_bpid_folder(BPID, docreate=False)
    # get json data
    with open(path + 'parents.json', "r") as read_file:
        parents = json.load(read_file)
    with open(path + 'children.json', "r") as read_file:
        children = json.load(read_file)
    bpid_list = []
    # extract BPIDs as list
    bpid_list = [BPID] # seed with main bpid
    for x in parents["parents"]: 
        bpid_list.append(x['institution']['id'])
        if x['parents']:  # just go one level up
            for z in x["parents"]: 
                bpid_list.append(z['institution']['id'])            
    for x in children:
        bpid_list.append(x['id'])
    print("Related BPIDs found: ", len(bpid_list), "=", str(bpid_list))
    return bpid_list



# eg
if False:
    for x in progress(BPIDS_LIST[:10]):
        api_extract_related_bpids(x)
        
# eg
if False:
    for x in progress(BPIDS_LIST[:10]):
        api_extract_related_bpids_from_local_file(x)

### 2.2 API: Extracting KBARTs

* for a single BPID, get nature/springer kbarts (books is postponed)
* save all in the main BPID folder (= customer the analysis focuses on)

In [21]:
url_books = "http://ml-read-online.live.sl.i.springer.com:7655/admin-portal/201802052054-134/kbart/book-report.xqy?bp={}"
url_journals = "http://ml-read-online.live.sl.i.springer.com:7655/admin-portal/201802052054-134/kbart/journal-report.xqy?bp={}"
url_nature = "http://entitlements-api.live.private.springernature.io/entitlements/v1/journals/kbart?bpid={}"


def api_extract_kbarts(BPID, BPID_TOP=None):
    if not BPID_TOP:
        BPID_TOP = BPID
    BPID, BPID_TOP = str(BPID), str(BPID_TOP)
    path = gen_bpid_folder(BPID_TOP)
    print("===\nExtracting KBARTs for", BPID, " -- top level=", BPID_TOP)
    if False:
        # not needed yet
        print("..books...")
        d = requests.get(url_books.format(BPID), auth=('casper-db-user', 'casper'))
        with open(path + BPID + "_books.kbart", 'wb') as outfile:
            outfile.write(d.text.encode('utf8'))
    #
    print("..journals/springer...")
    d = requests.get(url_journals.format(BPID), auth=('casper-db-user', 'casper'))
    with open(path + BPID + "_journals_springer.kbart", 'wb') as outfile:
        outfile.write(d.text.encode('utf8'))
    #
    print("..journals/nature...")
    d = requests.get(url_nature.format(BPID), auth=('casper-db-user', 'casper'))
    with open(path + BPID + "_journals_nature.kbart", 'wb') as outfile:
        outfile.write(d.text.encode('utf8'))

        
# eg
if False:
    for x in progress(BPIDS_LIST[:10]):
        api_extract_kbarts(x)


### 2.3 Finally - Running the Extraction

In [23]:
errors = []

for bpid in progress(BPIDS_LIST_UNIQUE):
    try:
        related_bpids = api_extract_related_bpids(bpid)
    except:
        related = []
        errors.append(bpid)
        print("++++ ERROR ++++")
        print("Errors index =", errors)
    for child in progress(related_bpids):
        api_extract_kbarts(child, bpid)

print("**** completed ****")
print("Errors index =", errors)

Checking Hierarchy for BPID:  3001049600 
..data saved in /Volumes/Albano/2020-08-06-holding-analysis-extraction/3001049600/
Related BPIDs found:  1 = ['3001049600']


===
Extracting KBARTs for 3001049600  -- top level= 3001049600
..journals/springer...
..journals/nature...

Checking Hierarchy for BPID:  2000022016 
..data saved in /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000022016/
Related BPIDs found:  8 = ['2000022016', '3000130459', '3000179129', '3000198352', '3000251517', '3991330314', '3001139967', '3002011430']


===
Extracting KBARTs for 2000022016  -- top level= 2000022016
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000130459  -- top level= 2000022016
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000179129  -- top level= 2000022016
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000198352  -- top level= 2000022016
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000251517  -- top level= 2000022016
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3991330314  -- top level= 2000022016
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3001139967  -- top level= 2000022016
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3002011430  -- top level= 2000022016
..journals/springer...
..journals/nature...

Checking Hierarchy for BPID:  3000201735 
..data saved in /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000201735/
Related BPIDs foun

===
Extracting KBARTs for 3000201735  -- top level= 3000201735
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000175179  -- top level= 3000201735
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 1000255885  -- top level= 3000201735
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 1600001227  -- top level= 3000201735
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 2000617610  -- top level= 3000201735
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000116918  -- top level= 3000201735
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000116931  -- top level= 3000201735
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000116934  -- top level= 3000201735
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000175175  -- top level= 3000201735
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000204988 

===
Extracting KBARTs for 3000161288  -- top level= 3000161288
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 3000675571  -- top level= 3000161288
..journals/springer...
..journals/nature...

Checking Hierarchy for BPID:  2000185358 
..data saved in /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000185358/
Related BPIDs found:  9 = ['2000185358', '3000130459', '3000251517', '3001070824', '3991330314', '3000092011', '3001139967', '3002011430', '3002114477']


...

===
Extracting KBARTs for 8200696305  -- top level= 8200696305
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 8200828607  -- top level= 8200696305
..journals/springer...
..journals/nature...

Checking Hierarchy for BPID:  8200150001 
..data saved in /Volumes/Albano/2020-08-06-holding-analysis-extraction/8200150001/
Related BPIDs found:  2 = ['8200150001', '8200828607']


===
Extracting KBARTs for 8200150001  -- top level= 8200150001
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 8200828607  -- top level= 8200150001
..journals/springer...
..journals/nature...

Checking Hierarchy for BPID:  3000121840 
..data saved in /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000121840/
Related BPIDs found:  2 = ['3000121840', '8200828607']


===
Extracting KBARTs for 3000121840  -- top level= 3000121840
..journals/springer...
..journals/nature...
===
Extracting KBARTs for 8200828607  -- top level= 3000121840
..journals/springer...
..journals/nature...

Checking Hierarchy for BPID:  2000349174 
..data saved in /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000349174/
Related BPIDs found:  1 = ['2000349174']


===
Extracting KBARTs for 2000349174  -- top level= 2000349174
..journals/springer...
..journals/nature...

Checking Hierarchy for BPID:  2000429567 
..data saved in /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000429567/
Related BPIDs found:  1 = ['2000429567']


===
Extracting KBARTs for 2000429567  -- top level= 2000429567
..journals/springer...
..journals/nature...


**** completed ****
Errors index = [3003695120, 30036927080, 3003343017, 2000341714, 8200769361, 3000633321]


## 3. Extracting Usage: Springer & Nature from GBQ

* requires a GBQ service account credentials (JSON file)

In [43]:
from google.cloud import bigquery

credentials_file = 'data_input/usage-data-reporting-523fb677bd0a.json'
project_id = 'usage-data-reporting'
# Explicitly use service account credentials by specifying the private key file
client_ref = bigquery.Client.from_service_account_json(credentials_file)

### 3.1 GBQ: Springer usage data 

* for a BPID, pull all usage data - Springer only for the moment
* if a BPID is not found or not valid, it simply outputs a zero-lines CSV

In [41]:

gbq_query = '''
Select DOI, Usage_2016, Usage_2017, Usage_2018, Denials_2016, Denials_2017, Denials_2018 FROM (
    SELECT 
    BP_ID, Item_DOI as DOI,  sum(clicks),
    sum(Case when Calendar_Year = 2016 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2018,
    sum(Case when Calendar_Year = 2016 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2016,
    sum(Case when Calendar_Year = 2017 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2017,
    sum(Case when Calendar_Year = 2018 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2018,

    FROM `usage-data-reporting.PROD_IFM_Usage.mv_usage_bp_monthly_spl` 
    Where 
    Document_Type = 'ARTICLE'
    and Item_DOI is not null
    and DOI is not null
    and clicks is not null
    and Calendar_Year in (2016, 2017, 2018)
    Group by 1, 2
    Order by 1, 2)
Where BP_ID = '{}' 
# limit 100
'''


def gbq_extract_usage_springer(BPID, BPID_TOP=None, skip_if_exists=True):
    "Extract springer usage via bpid-siteid mappings. Default: skip extraction if file already exists in destination"
    if not BPID_TOP:
        BPID_TOP = BPID
    BPID, BPID_TOP = str(BPID), str(BPID_TOP)
    path = gen_bpid_folder(BPID_TOP)
    print("===\nExtracting Springer Usage for", BPID, " -- top level=", BPID_TOP)
    #
    filename = path + BPID + "_usage_springer.csv"
    if os.path.exists(filename) and skip_if_exists:
        print("..skipping.. file exists:", filename)
    else:
        query_job = client_ref.query(gbq_query.format(BPID)).to_dataframe()
        print("..saving..", filename)
        query_job.to_csv(filename, index=False)


    

if False:
    for x in progress(BPIDS_LIST_UNIQUE[:1]):
        gbq_extract_usage_springer(x)        


errors = []

for b1 in progress(BPIDS_LIST_UNIQUE):
    try:
        related = extract_related_bpids_from_local_file(b1)
    except:
        related = []
        errors.append(b1)
        print("++++ ERROR ++++")
        print("Errors index =", errors)
    for b2 in progress(related):
        gbq_extract_usage_springer(b2, b1)

print("**** completed ****")
print("Errors index =", errors)


Checking Pre-saved Hierarchy for BPID:  3001049600 
Related BPIDs found:  1 = ['3001049600']


===
Extracting Springer Usage for 3001049600  -- top level= 3001049600
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3001049600/3001049600_usage_springer.csv

Checking Pre-saved Hierarchy for BPID:  2000022016 
Related BPIDs found:  8 = ['2000022016', '3000130459', '3000179129', '3000198352', '3000251517', '3991330314', '3001139967', '3002011430']


===
Extracting Springer Usage for 2000022016  -- top level= 2000022016
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000022016/2000022016_usage_springer.csv
===
Extracting Springer Usage for 3000130459  -- top level= 2000022016
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000022016/3000130459_usage_springer.csv
===
Extracting Springer Usage for 3000179129  -- top level= 2000022016
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000022016/3000179129_usage_springer.csv
===
Extracting Springer Usage for 3000198352  -- top level= 2000022016
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000022016/3000198352_usage_springer.csv
===
Extracting Springer Usage for 3000251517  -- top level= 2000022016
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000022016/3000251517_usage_springer.csv
===
Extracting Springer Usage for 3

===
Extracting Springer Usage for 3000201735  -- top level= 3000201735
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000201735/3000201735_usage_springer.csv
===
Extracting Springer Usage for 3000175179  -- top level= 3000201735
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000201735/3000175179_usage_springer.csv
===
Extracting Springer Usage for 1000255885  -- top level= 3000201735
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000201735/1000255885_usage_springer.csv
===
Extracting Springer Usage for 1600001227  -- top level= 3000201735
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000201735/1600001227_usage_springer.csv
===
Extracting Springer Usage for 2000617610  -- top level= 3000201735
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000201735/2000617610_usage_springer.csv
===
Extracting Springer Usage for 3

===
Extracting Springer Usage for 3000161288  -- top level= 3000161288
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000161288/3000161288_usage_springer.csv
===
Extracting Springer Usage for 3000675571  -- top level= 3000161288
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000161288/3000675571_usage_springer.csv

Checking Pre-saved Hierarchy for BPID:  2000185358 
Related BPIDs found:  9 = ['2000185358', '3000130459', '3000251517', '3001070824', '3991330314', '3000092011', '3001139967', '3002011430', '3002114477']


...

===
Extracting Springer Usage for 8200696305  -- top level= 8200696305
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/8200696305/8200696305_usage_springer.csv
===
Extracting Springer Usage for 8200828607  -- top level= 8200696305
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/8200696305/8200828607_usage_springer.csv

Checking Pre-saved Hierarchy for BPID:  8200150001 
Related BPIDs found:  2 = ['8200150001', '8200828607']


===
Extracting Springer Usage for 8200150001  -- top level= 8200150001
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/8200150001/8200150001_usage_springer.csv
===
Extracting Springer Usage for 8200828607  -- top level= 8200150001
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/8200150001/8200828607_usage_springer.csv

Checking Pre-saved Hierarchy for BPID:  3000121840 
Related BPIDs found:  2 = ['3000121840', '8200828607']


===
Extracting Springer Usage for 3000121840  -- top level= 3000121840
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000121840/3000121840_usage_springer.csv
===
Extracting Springer Usage for 8200828607  -- top level= 3000121840
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000121840/8200828607_usage_springer.csv

Checking Pre-saved Hierarchy for BPID:  2000349174 
Related BPIDs found:  1 = ['2000349174']


===
Extracting Springer Usage for 2000349174  -- top level= 2000349174
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000349174/2000349174_usage_springer.csv

Checking Pre-saved Hierarchy for BPID:  2000429567 
Related BPIDs found:  1 = ['2000429567']


===
Extracting Springer Usage for 2000429567  -- top level= 2000429567
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000429567/2000429567_usage_springer.csv


**** completed ****
Errors index = [3003695120, 30036927080, 3003343017, 2000341714, 8200769361, 3000633321]


### GBQ: 3.2 Nature usage data

* requires a GBQ service account credentials (JSON file)
* for each BPID, find the corresponding SITE_ID
* for a SITE_ID, pull all usage data - using Nature table


In [42]:
gbq_query = '''
Select DOI, Usage_2016, Usage_2017, Usage_2018, Denials_2016, Denials_2017, Denials_2018 FROM (

SELECT
Site_ID, Item_DOI as DOI,  sum(clicks),

sum(Case when Calendar_Year = 2016 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2016,
sum(Case when Calendar_Year = 2017 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2017,
sum(Case when Calendar_Year = 2018 and Access_YN = 'Y' Then Clicks Else 0 END) as Usage_2018,
sum(Case when Calendar_Year = 2016 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2016,
sum(Case when Calendar_Year = 2017 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2017,
sum(Case when Calendar_Year = 2018 and Access_YN = 'N' Then Clicks Else 0 END) as Denials_2018,

FROM `usage-data-reporting.PROD_IFM_Usage.mv_usage_bp_monthly_nat`
Where
Item_DOI is not null
and DOI is not null
and clicks is not null
and Calendar_Year in (2016, 2017, 2018)
Group by 1, 2
Order by 1, 2)

Where Site_ID = '{}'
'''


def extract_usage_nature(BPID, BPID_TOP=None, skip_if_exists=True):
    "Extract nature usage via bpid-siteid mappings. Default: skip extraction if file already exists in destination"
    if not BPID_TOP:
        BPID_TOP = BPID
    SITE_ID = bpid_to_siteid(int(BPID))
    BPID, BPID_TOP = str(BPID), str(BPID_TOP)
    path = gen_bpid_folder(BPID_TOP)
    if SITE_ID:
        print("===\nExtracting Nature Usage for", BPID, " -- top level=", BPID_TOP, "Site_id=", SITE_ID)
        #
        filename = path + BPID + "_usage_nature.csv"
        if os.path.exists(filename) and skip_if_exists:
            print("..skipping.. file exists:", filename)
        else:
            query_job = client_ref.query(gbq_query.format(str(SITE_ID))).to_dataframe()
            print("..saving..", filename)
            query_job.to_csv(path + BPID + "_usage_nature.csv", index=False)
    else:
        print("...mapping from", BPID, " to Site_id not found: skipping")
 
        
# eg
if False:
    for x in progress(BPIDS_LIST_UNIQUE[:10]):
        extract_usage_nature(x)


errors = []

for b1 in progress(BPIDS_LIST_UNIQUE):
    try:
        related = extract_related_bpids_from_local_file(b1)
    except:
        related = []
        errors.append(b1)
        print("++++ ERROR ++++")
        print("Errors index =", errors)
    # if BPIDs are found
    for b2 in progress(related):
        extract_usage_nature(b2, b1)

print("**** completed ****")
print("Errors index =", errors)

Checking Pre-saved Hierarchy for BPID:  3001049600 
Related BPIDs found:  1 = ['3001049600']


...mapping from 3001049600  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  2000022016 
Related BPIDs found:  8 = ['2000022016', '3000130459', '3000179129', '3000198352', '3000251517', '3991330314', '3001139967', '3002011430']


===
Extracting Nature Usage for 2000022016  -- top level= 2000022016 Site_id= 1950
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000022016/2000022016_usage_nature.csv
...mapping from 3000130459  to Site_id not found: skipping
...mapping from 3000179129  to Site_id not found: skipping
...mapping from 3000198352  to Site_id not found: skipping
...mapping from 3000251517  to Site_id not found: skipping
...mapping from 3991330314  to Site_id not found: skipping
...mapping from 3001139967  to Site_id not found: skipping
...mapping from 3002011430  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  3000201735 
Related BPIDs found:  43 = ['3000201735', '3000175179', '1000255885', '1600001227', '2000617610', '3000116918', '3000116931', '3000116934', '3000175175', '3000204988', '3000205002', '3000205076', '3000205082', '8200982076', '3004223722', '3004223724', '3001088854', '3001157578', '3001201143', '3004221684', '3004221752', '30042217

...mapping from 3000201735  to Site_id not found: skipping
===
Extracting Nature Usage for 3000175179  -- top level= 3000201735 Site_id= 4559
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000201735/3000175179_usage_nature.csv
...mapping from 1000255885  to Site_id not found: skipping
...mapping from 1600001227  to Site_id not found: skipping
...mapping from 2000617610  to Site_id not found: skipping
...mapping from 3000116918  to Site_id not found: skipping
...mapping from 3000116931  to Site_id not found: skipping
...mapping from 3000116934  to Site_id not found: skipping
...mapping from 3000175175  to Site_id not found: skipping
...mapping from 3000204988  to Site_id not found: skipping
...mapping from 3000205002  to Site_id not found: skipping
...mapping from 3000205076  to Site_id not found: skipping
...mapping from 3000205082  to Site_id not found: skipping
===
Extracting Nature Usage for 8200982076  -- top level= 3000201735 Site_id= 4665
..skip

===
Extracting Nature Usage for 3000161288  -- top level= 3000161288 Site_id= 4777
..skipping.. file exists: /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000161288/3000161288_usage_nature.csv
...mapping from 3000675571  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  2000185358 
Related BPIDs found:  9 = ['2000185358', '3000130459', '3000251517', '3001070824', '3991330314', '3000092011', '3001139967', '3002011430', '3002114477']


...

===
Extracting Nature Usage for 8200696305  -- top level= 8200696305 Site_id= 1661
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/8200696305/8200696305_usage_nature.csv
...mapping from 8200828607  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  8200150001 
Related BPIDs found:  2 = ['8200150001', '8200828607']


===
Extracting Nature Usage for 8200150001  -- top level= 8200150001 Site_id= 4984
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/8200150001/8200150001_usage_nature.csv
...mapping from 8200828607  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  3000121840 
Related BPIDs found:  2 = ['3000121840', '8200828607']


===
Extracting Nature Usage for 3000121840  -- top level= 3000121840 Site_id= 994
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/3000121840/3000121840_usage_nature.csv
...mapping from 8200828607  to Site_id not found: skipping

Checking Pre-saved Hierarchy for BPID:  2000349174 
Related BPIDs found:  1 = ['2000349174']


===
Extracting Nature Usage for 2000349174  -- top level= 2000349174 Site_id= 22319
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000349174/2000349174_usage_nature.csv

Checking Pre-saved Hierarchy for BPID:  2000429567 
Related BPIDs found:  1 = ['2000429567']


===
Extracting Nature Usage for 2000429567  -- top level= 2000429567 Site_id= 12883
..saving.. /Volumes/Albano/2020-08-06-holding-analysis-extraction/2000429567/2000429567_usage_nature.csv


**** completed ****
Errors index = [3003695120, 30036927080, 3003343017, 2000341714, 8200769361, 3000633321]


## 4. QAing the data

### identifiers info

In [51]:
print("BPIDs: ", len(BPIDS_LIST)) 
print("BPIDs unique: ", len(BPIDS_LIST_UNIQUE)) 
print("SiteIDs: ", len(siteids)) 
print("BPIDs (unique) mapped to SiteIDs: ", count_bpid_mapped())   

BPIDs:  192
BPIDs unique:  189
SiteIDs:  9699
BPIDs (unique) mapped to SiteIDs:  151


### combined table BPIDs + SiteIDs

In [92]:
bpids['SITEID'] = bpids.apply(lambda x: str(bpid_to_siteid(x['BPID'])), axis=1)
bpids.to_csv("data_output/bpids_siteids_combined.csv", index=False)
bpids

,BPID,Account,Contact,SITEID
0,3000612920,Research Organization of Information,Martijn Timman,None
1,2000703964,INSERM DISC-IST,Martijn Timman,None
2,2000478521,The University of Tokyo,Martijn Timman,4604
3,2000277680,Shanghai Library / Institute of,Martijn Timman,14945
4,2000342845,National Library of China NLC,Martijn Timman,12245
...,...,...,...,...
187,3000142567,Wesleyan University,Dylan Moulton,1558
188,8200178197,Wheaton College,Dylan Moulton,9660
189,8200699471,Williams College,Dylan Moulton,4474
190,8200711893,Worcester Polytechnic Institute,Dylan Moulton,1566


### folders created (from IDs) 

In [46]:
path = "/Volumes/Albano/2020-08-06-holding-analysis-extraction"

In [54]:
folders = [f for f in os.listdir(path) if not f.startswith('.')]
print("Folders in output: ", len(folders))
difference = [x for x in BPIDS_LIST_UNIQUE if str(x)  not in folders]
print("BPIDs missing from folders output:", difference)

Folders in output:  189
BPIDs missing from folders output: []


### folders that don't contain any BPIDs hierarchy info 

that's because the API calls above failed (section 1)

In [93]:
path = "/Volumes/Albano/2020-08-06-holding-analysis-extraction"
folders = [f for f in os.listdir(path) if not f.startswith('.')]
errors = []
for f in folders:
    if not os.path.exists(path+"/"+f+"/children.json") and not os.path.exists(path+"/"+f+"/parents.json"):
        errors.append(f)
        
print(errors)

['2000341714', '3000633321', '3003343017', '30036927080', '3003695120', '8200769361']


memo: compress results at the end with 

```
tar -zcvf 2020-08-06-holding-analysis-extraction.tar.qz 2020-08-06-holding-analysis-extraction
```